# sqlite3

In [11]:
import sqlite3

In [12]:
conn = sqlite3.connect('mydb.db')
cursor = conn.cursor()

In [ ]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS sample_data (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER
    )
''')
cursor.execute("INSERT INTO sample_data (name, age) VALUES ('Alice', 25)")
cursor.execute("INSERT INTO sample_data (name, age) VALUES ('Bob', 30)")
cursor.execute("INSERT INTO sample_data (name, age) VALUES ('Charlie', 22)")

In [13]:
conn.commit()
conn.close()

# SqlAlchemy

In [18]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect

In [19]:
engine = create_engine('sqlite:///mydb.db')

## Inspector

In [20]:
inspector = inspect(engine)

In [21]:
schemas = inspector.get_schema_names()
schemas

['main']

In [22]:
schema = schemas[0]
table_names = inspector.get_table_names(schema=schema)
table_names

[]

In [23]:
table_name = table_names[0]
columns = inspector.get_columns(table_name, schema=schema)
columns

IndexError: list index out of range

In [24]:
url = "https://raw.githubusercontent.com/ywchiu/riii/master/data/house-prices.csv"
df = pd.read_csv(url)
df

,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East
...,...,...,...,...,...,...,...,...
123,124,119700,1900,3,3,3,Yes,East
124,125,147900,2160,4,3,3,Yes,East
125,126,113500,2070,2,2,2,No,North
126,127,149900,2020,3,3,1,No,West
